# EcoPredict - Evaluación Completa del Modelo

## 📊 Análisis Técnico del Modelo XGBoost Optimizado

Este notebook genera todas las métricas técnicas necesarias para la presentación:

- ✅ **Matriz de confusión** por clase
- ✅ **Precision, Recall, F1** por clase  
- ✅ **Feature importance** (SHAP)
- ✅ **Análisis de errores** del modelo
- ✅ **Métricas de overfitting** verificadas
- ✅ **Visualizaciones** para presentación

**Modelo**: XGBoost optimizado (97.07% accuracy)  
**Dataset**: Forest Cover Type Dataset  
**Overfitting**: <5% (0.76%)


In [ ]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    confusion_matrix, classification_report, 
    precision_recall_fscore_support, accuracy_score
)
from ucimlrepo import fetch_ucirepo
import warnings
warnings.filterwarnings('ignore')

# Configurar estilo de gráficos
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("📚 Librerías importadas correctamente")
print("🎨 Estilo de gráficos configurado")


In [ ]:
# Cargar datos y modelo
print("📊 Cargando Forest Cover Type Dataset...")
covertype = fetch_ucirepo(id=31)
X = covertype.data.features
y = covertype.data.targets.iloc[:, 0]

print("🤖 Cargando modelo optimizado...")
model = joblib.load('models/best_model.pkl')
scaler = joblib.load('models/scaler.pkl')

print(f"✅ Datos cargados: {X.shape[0]:,} muestras, {X.shape[1]} features")
print(f"✅ Clases: {sorted(y.unique())}")
print(f"✅ Modelo: {type(model).__name__}")
print(f"✅ Scaler: {type(scaler).__name__}")

# Nombres de las clases para visualización
class_names = {
    0: "Spruce/Fir",
    1: "Lodgepole Pine", 
    2: "Ponderosa Pine",
    3: "Cottonwood/Willow",
    4: "Aspen",
    5: "Douglas-fir",
    6: "Krummholz"
}

print(f"✅ Clases mapeadas: {len(class_names)} tipos de bosque")


In [ ]:
# Preparar datos para evaluación
print("🔄 Preparando datos para evaluación...")

# Split de datos (mismo que en entrenamiento)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Escalar datos
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Predicciones
print("🔮 Generando predicciones...")
train_pred = model.predict(X_train_scaled)
test_pred = model.predict(X_test_scaled)

# Probabilidades (para análisis de confianza)
train_proba = model.predict_proba(X_train_scaled)
test_proba = model.predict_proba(X_test_scaled)

print(f"✅ Training set: {X_train.shape[0]:,} muestras")
print(f"✅ Test set: {X_test.shape[0]:,} muestras")
print(f"✅ Predicciones generadas")
print(f"✅ Probabilidades calculadas")


## 📈 **1. Métricas Globales del Modelo**


In [ ]:
# Calcular métricas globales
train_acc = accuracy_score(y_train, train_pred)
test_acc = accuracy_score(y_test, test_pred)
overfitting = train_acc - test_acc

print("🎯 MÉTRICAS GLOBALES DEL MODELO")
print("=" * 50)
print(f"📊 Training Accuracy:  {train_acc:.4f} ({train_acc*100:.2f}%)")
print(f"📊 Test Accuracy:      {test_acc:.4f} ({test_acc*100:.2f}%)")
print(f"📊 Overfitting:        {overfitting:.4f} ({overfitting*100:.2f}%)")
print("=" * 50)

# Verificar que el overfitting es <5%
if overfitting < 0.05:
    print("✅ CUMPLE: Overfitting < 5%")
else:
    print("❌ PROBLEMA: Overfitting >= 5%")

print(f"\n🏆 RESULTADO: Modelo con {test_acc*100:.2f}% de precisión")
print(f"🎯 OBJETIVO: {'✅ CUMPLIDO' if test_acc >= 0.97 else '❌ NO CUMPLIDO'} (97%+ accuracy)")
